# Modeling: Подготовка датасетов для моделей и создание самих моделей

## 1. Импорт библиотек

Импортируем стандартные библиотеки для анализа данных, а также пользовательские функции для визуализации и первичного осмотра данных.

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

try:
    from utils.paths import join_path as pj
except ImportError:
    from scooter_sharing_analysis.utils.paths import join_path as pj

## 2. Загрузка очищенных данных

Загружаем файл rides_weather_data.csv нормализованный после очистки.

In [ ]:
# Настраиваем словари с параметрами чтения CSV: путь, кодировка, дата-колонки
rides_options = {
    "filepath_or_buffer": pj("data", "rides_weather_data.csv"),
    "encoding": "utf-8",
    "parse_dates": ["start_date", "end_date", "day_timestamp", "hour_timestamp"],
}

rides_weather_data = pd.read_csv(**rides_options)

## 3. Создание факторов для моделей: временные, лаговые, погодные

## 3.1. Создание временных и лаговых факторов

Группируем данные по часовому ряду, получая спрос за каждый час (demand), добавляет новую колонку: 'hour_of_day'. Делим данные на тестовые и тренировочные с соотношение 1:4, добавляем лаговые колонки: 'lag_1h', 'lag_24h', 'mean_last_24h', 'mean_last_7d'

In [ ]:
# Группировка данных и получение спроса
demand = rides_weather_data.groupby("hour_timestamp").agg({"id": "count"}).rename(columns={"id": "demand"}).reset_index()

# Добавление новых колонки на основе "hour_timestamp"
demand["hour_of_day"] = demand["hour_timestamp"].dt.hour

# Создание лаговых признаков
demand["lag_1h"] = demand["demand"].shift(1)
demand["lag_24h"] = demand["demand"].shift(24)
demand["mean_last_24h"] = demand["lag_1h"].rolling(24).mean()
demand["mean_last_7d"] = demand["lag_1h"].rolling(24 * 7).mean()

# Удаление строк с Nan значениями в лагах
demand = demand.dropna()

# Разбиение данных на тестувую и тренировочную части
X_demand_train, X_demand_test, Y_demand_train, Y_demand_test = train_test_split(demand, demand["demand"], test_size=0.2, shuffle=False)

# Вывод первой 1000 строк
X_demand_train.head(1000)


## 3.2. OHE кодирование категориальных признаков: start_location, promo

Создаем модель OHE для кодирования категориальных признаков "start_location", "promo", получаем имена будущих колонок для датафрейма с закодированными данными, на основе закодированных данных и имен колонок создаем новый датафрейм, который объединяем по оси 1 с начальным датафреймом предварительно удалив уже ненужные категориальные признаки "start_location", "promo"

In [ ]:
# Создание OHE модели с заданными параметрами
encoder = OneHotEncoder(drop="first", sparse_output=False)

# Название колонок с категориальными признаками
column_names = ["start_location", "promo"]

# Кодирование категориальных признаков на основе модели
encoded_array = encoder.fit_transform(rides_weather_data[column_names])

# Получение названий колонок для будущего датафрейма
feat_names = encoder.get_feature_names_out(column_names)

# Создание датафрейма с закодированными признаками
encoded_data = pd.DataFrame(encoded_array, columns=feat_names)

# Объединение датафреймов по 1 оси
encoded_rides_weather_data = pd.concat([rides_weather_data.drop(columns=column_names, axis=1), encoded_data], axis=1)

# Вывод получившегося датафрейма
encoded_rides_weather_data